In [7]:
import os
import pyodbc
import pandas as pd

In [52]:
#loading the estimates data 
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=estimates;'
                      'Trusted_Connection=yes;')

hhpop_2020 ="SELECT [mgra_id], [yr_id], [housing_type_id], [population]"\
                  "FROM [estimates].[est_2020_01].[dw_population]"\
                   "WHERE housing_type_id= 1"
                  
             
hhpop = pd.read_sql_query(hhpop_2020, conn)    

print(hhpop.head())

uniqueyr = hhpop['yr_id'].nunique()
print(uniqueyr)

      mgra_id  yr_id  housing_type_id  population
0  1400000101   2010                1          41
1  1400000102   2010                1           0
2  1400000201   2010                1          74
3  1400000202   2010                1           0
4  1400000301   2010                1         126
11


In [34]:
# loading the dim_mgra_denormalized table from demographic warehouse 
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

dim_mgra ="SELECT *"\
                  "FROM [demographic_warehouse].[dim].[mgra_denormalize]"\
                   "WHERE series= 14 " 
             
dim_mgra = pd.read_sql_query(dim_mgra, conn)    

dim_mgra.dtypes


mgra_id             int64
mgra                int64
series              int64
region             object
sra                object
tract              object
supervisorial      object
college            object
cpa                object
jurisdiction       object
zip                object
secondary          object
elementary         object
msa                object
sdcouncil          object
transit            object
unified            object
jurisdiction_id     int64
cpa_id              int64
taz                 int64
dtype: object

In [25]:
# setting up the query string for loading the mgra data from geodepot

mgra_query = """SELECT  [OBJECTID]
      ,[MGRA]
      ,[CT10]
      ,[CT10BG]
      ,[SRA]
      ,[MSA]
      ,[City]
      ,[ZIP]
      ,[Sphere]
      ,[CPA]
      ,[CPASG]
      ,[TAZ]
      ,[Council]
      ,[Super]
      ,[LUZ]
      ,[Elem]
      ,[Unif]
      ,[High]
      ,[Coll]
      ,[Transit]
      ,[Shape] """ + \
    ".STGeometryN(1).ToString()"  + "FROM [GeoDepot].[gis].[MGRA13PT]"

 # + geo_col + '.STGeometryN(1).ToString()'  
 #  ,[Shape]
   #-  ,[GDB_GEOMATTR_DATA]

In [37]:
# loading the mgra data from geodepot

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=sql2014b8;'
                      'Database=GeoDepot;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
             
mgra = pd.read_sql_query(mgra_query, conn)    

mgra.dtypes


OBJECTID     int64
MGRA         int64
CT10         int64
CT10BG       int64
SRA          int64
MSA          int64
City         int64
ZIP          int64
Sphere       int64
CPA          int64
CPASG        int64
TAZ          int64
Council      int64
Super        int64
LUZ          int64
Elem         int64
Unif         int64
High         int64
Coll         int64
Transit      int64
            object
dtype: object

In [56]:
# creating cross-walking between the age estimates table, dim_mgra table, and mgra geodepot file

mgra_shape= mgra.merge(dim_mgra, left_on='MGRA', right_on='mgra')
mgra_shape.shape[0]
mgra_shape.head()
estimates_mgra= mgra_shape.merge(hhpop, left_on= 'mgra_id', right_on= 'mgra_id')
print(estimates_mgra.shape[0])
estimates_mgra

# checking the number of rows in the final df remains intact during merge

estimates_mgra.shape[0]== uniqueyr*27316

300476


True

In [ ]:
from arcgis.gis import GIS

gis = GIS()

In [6]:
gis

GIS @ https://www.arcgis.com version:9.1

In [ ]:
# pull data from geodepot


# series 13 for geodepot 